# Requirements

python 3.9

pip install pandas

pip install openpyxl

# Notes

* Principal claimants only (one case per family)
* Only includes positive/negative/abandoned/withdrawn (thus excludes vacation cases, excludes administratively decided, etc)
* Only includes most recent decision for a given IRB_file_no (i.e. if denied, then JR, then granted only catches granted)




## Setup

In [1]:
#import packages
import pandas as pd
import numpy as np


In [2]:
#set paths to data
RPDCases13_21 = 'CONFIDENTIAL_DATA//ATIP A-2021-01792 - List 1.xlsx'
RPDClaimTypes = 'CONFIDENTIAL_DATA//ClaimTypes.xlsx'
RPDOutputs = 'CONFIDENTIAL_DATA//CleanedData.json'
TableDir = 'OUTPUTS//'

In [3]:
#set options
pd.options.display.max_rows = 10
FirstYear = 2013
LastYear = 2021

## Helper Functions

In [4]:
# function to rename columns to match 2019 labels
def rename_columns(df):
    df=df.rename(columns={
        'file_no': 'IRB_file_no',
        'file no': 'IRB_file_no',
        'principal_indicator': 'Principal_Indicator',
        'principal indicator': 'Principal_Indicator',
        'date_referred':'Date_Referred',
        'date referred':'Date_Referred',
        'decision_date': 'Decision_Date',
        'decision date': 'Decision_Date',
        'disposition_type_desc_e':'Disposition_Type',
        'disposition type desc e':'Disposition_Type',
        'Disposition':'Disposition_Type',
        'explanation_desc_e': 'Explanation',
        'explanation desc e': 'Explanation',
        'country_persec_desc_e': 'Country_Persecution',
        'country persec desc e': 'Country_Persecution',
        'basis category e':'Category',
        'basis type e':'Type'})
    return(df)

#Note can add new column names. No problem if not exist in other years, b/c will ignore if not exist

In [5]:
#function to clean up decisions dataframes
def clean_df(df):

    #remove space before and after column names, replace spaces within with _, remove accents
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

    # Remove any non final decisions
    df = df.drop(df[df['Disposition_Type'] != 'Final Decision'].index)

    #Remove any non principal claimants
    df = df.drop(df[df['Principal_Indicator'] != 'Y'].index)

    #Remove unecessary cases (e.g. removes deceased, vacation, cessation, administrative, etc)
    indexNames = df[(df['Explanation'] == 'Positive')
                    | (df['Explanation'] == 'Positive-File-Review Process (Paper)')
                    | (df['Explanation'] == 'Negative')
                    | (df['Explanation'] == 'Expedited Positive')
                    | (df['Explanation'] == 'Neg. No Cred Basis')
                    | (df['Explanation'] == 'Abandoned')
                    | (df['Explanation'] == 'Withdrawn')
                    ].index
    df = df.loc[indexNames]
    
    #Make dates usable
    df['Date_Referred'] = pd.to_datetime(df['Date_Referred'])
    df['Decision_Date'] = pd.to_datetime(df['Decision_Date'])

    #Remove legacy claims (i.e. referred pre dec 15 2012 or no info on referred)
    df.drop(df[df['Date_Referred'] < pd.Timestamp(2012, 12, 15)].index, inplace=True)
    df = df[df['Date_Referred'].notna()]
    
    #Remove Decision Dates before FirstYear after LastYear or no info on decision date
    df.drop(df[df['Decision_Date'] < pd.Timestamp(FirstYear, 1, 1)].index, inplace=True)
    df.drop(df[df['Decision_Date'] > pd.Timestamp(LastYear, 12, 31)].index, inplace=True)
    df = df[df['Decision_Date'].notna()]
        
    #Replace NAN in pandas with np NAN for easier calulcation
    #df=df.replace(pd.NA, np.nan)
    
    #Shorten Congo name for easier viewing
    df=df.replace('Congo, Democratic Republic', 'Congo, DRC')
    
    #Replace blank countires with Unknown
    df['Country_Persecution'] = df['Country_Persecution'].fillna('Unknown')

    #Remove unwanted columns
    df = df[['IRB_file_no','Date_Referred','Decision_Date', 'Explanation', 'Country_Persecution']]
    
    #Add numeric columns
    df["Positive"]=np.where((df['Explanation']=='Positive')|(df['Explanation']=='Expedited Positive')|(df['Explanation']=='Positive-File-Review Process (Paper)'), 1, 0)
    df["Negative"]=np.where((df['Explanation']=='Negative')|(df['Explanation']=='Neg. No Cred Basis'), 1, 0)
    df["Abandoned_Withdrawn"]=np.where((df['Explanation']=='Abandoned')|(df['Explanation']=='Withdrawn'), 1, 0)     

    #Reset index
    df = df.reset_index(drop=True)

    return(df)

In [6]:
#function to include only merits
def Only_Merits(df):
    indexNames = df[(df['Explanation'] == 'Positive')
        | (df['Explanation'] == 'Positive-File-Review Process (Paper)')
        | (df['Explanation'] == 'Negative')
        | (df['Explanation'] == 'Expedited Positive')
        | (df['Explanation'] == 'Neg. No Cred Basis')
        ].index
    df = df.loc[indexNames]
    
    return(df)

In [7]:
#function to get years in df
def get_years(df):
    years = df['Decision_Date'].unique()
    yearlist = list(years)
    yearlist.sort()
    yearlist =yearlist
    
    return(yearlist)

In [8]:
#function to round a column in a database to a given number
def custom_round(x, base):
    return int(base * round(float(x)/base))

def round_df(df,list_of_cols, base=20):
    for c in list_of_cols:
        df[c] = df[c].apply(lambda x: custom_round(x, base))
    return df

In [9]:
#function to save to csv, with good formatting
def save_to_csv(target_df, name_file):
    target_df.columns=target_df.columns.str.replace('_',' ')
    target_df.columns=target_df.columns.str.replace('Recognition Rate','Recognition Rate (%)')
    target_df.columns=target_df.columns.str.replace('Proportion','Proportion (%)')
    target_df.to_csv(name_file)
    

## Load Data

In [10]:
#get the 2013-2021 cases from A-2021-01792 skipping the heading section
df13_21 = pd.read_excel(RPDCases13_21, skiprows=4)

# print column names
print(df13_21.columns)

print(len(df13_21))




Index(['file no', 'Post CIF indicator', 'disposition type desc e',
       'explanation desc e', 'sub explanation desc e', 'decision date',
       'date decision mailed', 'principal indicator', 'language of record',
       'date referred', 'minister counsel participation type e',
       'member fullname', 'country persec desc e', 'associated fileno list',
       'counsel fullname', 'manifestly unfounded claim indicator',
       'stream type list e'],
      dtype='object')
163893


In [11]:
#get claim types (from 2021)
dfClaimTypes = pd.read_excel(RPDClaimTypes,sheet_name='List')

# print column names
print(dfClaimTypes.columns)
print(len(dfClaimTypes))

Index(['file no', 'basis category e', 'basis type e', 'Post CIF indicator'], dtype='object')
342773


## Clean Data

In [12]:
#match the key column names to the df19 format
df13_21 = rename_columns(df13_21)


In [13]:
#clean the outcomes dataframes
df13_21 = clean_df(df13_21)


In [14]:
#clean the claim types dataframe
dfClaimTypes=rename_columns(dfClaimTypes)
dfClaimTypes.drop_duplicates(inplace=True)
dfClaimTypes.dropna(subset=['Category'],inplace=True )
dfClaimTypes.drop('Post CIF indicator', inplace=True, axis=1)
dfClaimTypes['Category']=dfClaimTypes['Category'].str.replace('–','-')
dfClaimTypes['Type']=dfClaimTypes['Type'].str.replace('–','-')
dfClaimTypes.reset_index(inplace=True, drop=True)

# print column names
print(dfClaimTypes.columns)
print(len(dfClaimTypes))

Index(['IRB_file_no', 'Category', 'Type'], dtype='object')
312631


# Combine datasets

In [15]:
#combine outcome dataframes to dfAllYears
#dfAllYears = pd.concat([df13_18, df19, df20, df21], ignore_index=True)
dfAllYears=df13_21

print('Number of rows in all years dataset: '+str(len(dfAllYears)))
#Remove full duplicates
dfAllYears.drop_duplicates(inplace=True)

#Sort
dfAllYears=dfAllYears.sort_values(by=['Decision_Date'], ascending=False).reset_index(drop=True)

#Remove duplicate IRB_file_number, keeping the most recent one only
dfAllYears = dfAllYears.drop_duplicates(subset=['IRB_file_no'], keep=False).reset_index(drop=True)

#Simplify dates to years
dfAllYears['Date_Referred'] = pd.DatetimeIndex(dfAllYears['Date_Referred']).year
dfAllYears['Decision_Date'] = pd.DatetimeIndex(dfAllYears['Decision_Date']).year


print('Number of unique IRB_file_nums in all years dataset: '+str(dfAllYears['IRB_file_no'].nunique()))

# print column names
print(dfAllYears.columns)

Number of rows in all years dataset: 119883
Number of unique IRB_file_nums in all years dataset: 113004
Index(['IRB_file_no', 'Date_Referred', 'Decision_Date', 'Explanation',
       'Country_Persecution', 'Positive', 'Negative', 'Abandoned_Withdrawn'],
      dtype='object')


In [16]:
#Combine the decisions and claim types dfs
tempdf = dfAllYears.copy()
dfCombined=tempdf.merge(dfClaimTypes,how='left',on='IRB_file_no')
dfCombined['Category']=dfCombined['Category'].replace(np.nan,'No Category Provided')
dfCombined['Type']=dfCombined['Type'].replace(np.nan,'No Type Provided')
print('Number of rows in combined dataset: '+str(len(dfCombined)))
print('Number of unique IRB_file_nums in combined dataset: '+str(dfCombined['IRB_file_no'].nunique()))

# print column names
print(dfCombined.columns)


Number of rows in combined dataset: 140129
Number of unique IRB_file_nums in combined dataset: 113004
Index(['IRB_file_no', 'Date_Referred', 'Decision_Date', 'Explanation',
       'Country_Persecution', 'Positive', 'Negative', 'Abandoned_Withdrawn',
       'Category', 'Type'],
      dtype='object')


## Pivot Table Functions

In [17]:
#Function to summarize single level

def summarize_one_level(df,col_wanted='Category',row_nums=40, round_nearest=20, fewer_cols=True):

    #Remove duplicates
    tempdf1=df.copy(deep=True)
    tempdf1.drop_duplicates(subset=['IRB_file_no',col_wanted], keep='first', inplace=True)
    
    #Create pivot tabel
    piv =pd.pivot_table(tempdf1, values=['Positive', 'Negative', 'Abandoned_Withdrawn'], index=[col_wanted], 
                   margins = True,aggfunc = np.sum)

    #Avoid double counting in All
    tempdf=df.copy(deep=True)
    tempdf.drop_duplicates(subset=['IRB_file_no'], keep='first', inplace=True)
    piv.at['All','Abandoned_Withdrawn']=tempdf['Abandoned_Withdrawn'].sum()
    piv.at['All','Negative']=tempdf['Negative'].sum()
    piv.at['All','Positive']=tempdf['Positive'].sum()

    #Add All category
    piv['All']=piv.sum(axis=1)
    piv=piv.sort_values(by='All', ascending=False)
   
    #Add proporition
    piv['Proportion']=100*piv['All']/piv.at['All','All']
    piv=round_df(piv,['Proportion'],1)

    #Add recognition rates
    piv['Recognition_Rate']=100*piv['Positive']/(piv['Positive']+piv['Negative'])
    
    #Round to nearest -- if 1 do not round
    piv=round_df(piv,['Abandoned_Withdrawn','Negative','Positive','All'],round_nearest)
    
    #Round to whole numer  - Need to remove a small number of files with NAN as Recognition Rate b/c only abandoned
    try:
        piv = piv.dropna(subset=['Recognition_Rate'])
        piv=round_df(piv,['Recognition_Rate'],1)
    except:
        print('Could not round')
    
    # remove rows where 'All' = 0
    piv = piv[piv['All'] != 0]

    # if fewer_cols is True, drop cols "Abandoned_Withdraw","Negative","Positive"
    if fewer_cols:
        piv.drop(columns=["Abandoned_Withdrawn","Negative","Positive"], inplace=True)
    
    #change display options
    pd.options.display.max_rows = row_nums

    return(piv)   

In [18]:
#Function to summarize two levels

def summarize_two_level(df,
                        col1_wanted='Category',
                        var_col1_wanted='Particular Social Group – Sexual Orientation',
                        col2_wanted='Type',
                        row_nums=40,
                        round_nearest=20,
                        fewer_cols=True):

    #filter for first variable
    tempdf1=df.copy(deep=True)
    tempdf1=tempdf1.drop(tempdf1[tempdf1[col1_wanted] != var_col1_wanted].index)
    
    #Remove duplicates
    tempdf1.drop_duplicates(subset=['IRB_file_no',col1_wanted,col2_wanted], keep='first', inplace=True)
    
    #Create pivot tabel
    piv =pd.pivot_table(tempdf1, values=['Positive', 'Negative', 'Abandoned_Withdrawn'], index=[col1_wanted,col2_wanted], 
                   margins = True,aggfunc = np.sum)

    #Avoid double counting in All
    tempdf2=df.copy(deep=True)
    tempdf2=tempdf2.drop(tempdf2[tempdf2[col1_wanted] != var_col1_wanted].index)
    tempdf2.drop_duplicates(subset=['IRB_file_no',col1_wanted], keep='first', inplace=True)
    piv.at['All','Abandoned_Withdrawn']=tempdf2['Abandoned_Withdrawn'].sum()
    piv.at['All','Negative']=tempdf2['Negative'].sum()
    piv.at['All','Positive']=tempdf2['Positive'].sum()
    
    #Add All row for totals
    piv['All']=piv.sum(axis=1)
    piv=piv.sort_values(by='All', ascending=False)
          
    #Rename All row
    piv = piv.rename(index={'All': var_col1_wanted})
    piv = piv.rename(index={'': 'All'})
    
    #Add proporition
    piv['Proportion']=100*piv['All']/piv.loc[var_col1_wanted,'All'].loc['All']
    piv=round_df(piv,['Proportion'],1)
    
    #Add recognition rates
    piv['Recognition_Rate']=100*piv['Positive']/(piv['Positive']+piv['Negative'])

    #Round to nearest -- if 1 do not round
    piv=round_df(piv,['Abandoned_Withdrawn','Negative','Positive','All'],round_nearest)
    
    #Round to whole numer  - Need to remove a small number of files with NAN as Recognition Rate b/c only abandoned
    try:
        piv = piv.dropna(subset=['Recognition_Rate'])
        piv=round_df(piv,['Recognition_Rate'],1)
    except:
        print('Could not round')
    
    # remove rows where 'All' = 0
    piv = piv[piv['All'] != 0]

    # if fewer_cols is True, drop cols "Abandoned_Withdraw","Negative","Positive"
    if fewer_cols:
        piv.drop(columns=["Abandoned_Withdrawn","Negative","Positive"], inplace=True)
    
    #change display options
    pd.options.display.max_rows = row_nums

    return(piv)   

## Summary 

In [19]:
#count
print('Number of rows: '+str(len(dfCombined)))
print('')
print('Number of unique IRB_file_nums: '+str(dfCombined['IRB_file_no'].nunique()))
print('')
print('years:')
print(get_years(dfCombined))

# print column names
print(dfCombined.columns)


Number of rows: 140129

Number of unique IRB_file_nums: 113004

years:
[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Index(['IRB_file_no', 'Date_Referred', 'Decision_Date', 'Explanation',
       'Country_Persecution', 'Positive', 'Negative', 'Abandoned_Withdrawn',
       'Category', 'Type'],
      dtype='object')


## Tables for Paper on Claim Types

In [20]:
# Prepare data by aggregating categories
dfAggregated=dfCombined.copy()
dfAggregated['Aggregated_Category']=dfAggregated['Category']

#aggergate no nexus
aggregation_list=['No Nexus - Criminality/Corruption','No Nexus - Varied/Other']
aggregation_list='|'.join(aggregation_list)
dfAggregated['Aggregated_Category']=dfAggregated['Aggregated_Category'].str.replace(aggregation_list,'No Nexus', regex=True)

#aggregate political opinion
aggregation_list=[
    'Political Opinion - Activity/Occupation',
    'Political Opinion - Organization',
    'Political Opinion - Varied/Other',
    'Political Opinion - Military Service',
    'Political Opinion - State Policy Issues',
    'Political Opinion - Activism']
aggregation_list='|'.join(aggregation_list)
dfAggregated['Aggregated_Category']=dfAggregated['Aggregated_Category'].str.replace(aggregation_list,'Political Opinion', regex=True)

#aggregate PSG
aggregation_list=[
    'Particular Social Group - Gender-based/Domestic Violence',
    'Particular Social Group - Sexual Orientation',
    'Particular Social Group - Varied/Other']
aggregation_list='|'.join(aggregation_list)
dfAggregated['Aggregated_Category']=dfAggregated['Aggregated_Category'].str.replace(aggregation_list,'Particular Social Group', regex=True)



In [21]:
# Table 1: Per year

piv=summarize_one_level(dfCombined,'Decision_Date',row_nums=20, round_nearest=20, fewer_cols=False)
piv.drop('Proportion', inplace=True, axis=1)
save_to_csv(piv, TableDir + 'Table 1 per year.csv')
piv

,Abandoned Withdrawn,Negative,Positive,All,Recognition Rate (%)
Decision_Date,,,,,
All,10360,31240,71400,113000,70
2021,3020,6280,17000,26320,73
2019,2100,5600,13280,20980,70
2020,1160,3380,8740,13280,72
2018,1540,4000,7100,12640,64
2017,800,3520,8020,12360,69
2016,540,2560,5840,8960,70
2015,400,2260,4880,7540,68
2014,400,2340,4300,7040,65


In [22]:
# Table 1a: Countries

piv=summarize_one_level(dfCombined,'Country_Persecution',row_nums=500, round_nearest=20)
save_to_csv(piv, TableDir + 'Table 1a countries.csv')
piv

,All,Proportion (%),Recognition Rate (%)
Country_Persecution,,,
All,113000,100,70
Nigeria,9100,8,50
Haiti,7240,6,41
China,6820,6,56
Iran,5960,5,95
Pakistan,4880,4,76
Mexico,4740,4,36
Turkey,4700,4,92
India,4480,4,38


In [23]:
# Table 2: Categories (aggregated)

piv=summarize_one_level(dfAggregated,'Aggregated_Category',row_nums=20, round_nearest=20)
save_to_csv(piv, TableDir + 'Table 2 categories.csv')
piv

,All,Proportion (%),Recognition Rate (%)
Aggregated_Category,,,
All,113000,100,70
Political Opinion,38740,34,78
Particular Social Group,31280,28,72
No Nexus,25580,23,48
Religion,18580,16,78
Race/Ethnicity/Nationality,13580,12,79
No Category Provided,4480,4,68


In [24]:
# Table 2a: Categories (unaggregated)

piv=summarize_one_level(dfAggregated,'Category',row_nums=20, round_nearest=20)
save_to_csv(piv, TableDir + 'Table 2a categories unaggregated.csv')
piv

,All,Proportion (%),Recognition Rate (%)
Category,,,
All,113000,100,70
No Nexus - Criminality/Corruption,22420,20,48
Political Opinion - Activity/Occupation,20120,18,80
Religion,18580,16,78
Particular Social Group - Gender-based/Domestic Violence,14300,13,70
Race/Ethnicity/Nationality,13580,12,79
Particular Social Group - Sexual Orientation,12760,11,77
Political Opinion - Organization,11460,10,76
Particular Social Group - Varied/Other,5660,5,65


In [25]:
#Table 3 Political Types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Aggregated_Category',
                        var_col1_wanted='Political Opinion',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
#If you want to save
save_to_csv(piv, TableDir + 'Table 3 PO types.csv')
piv

All  \
Aggregated_Category Type                                                        
Political Opinion   All                                                 38740   
                    Varied/Other                                        12520   
                    Anti-government                                     11740   
                    NGO/Community worker                                 1740   
                    Hizmet (Gulen) Movement                              1700   
                    State employee/representative                        1180   
                    Journalist                                            980   
                    Evasion                                               760   
                    Family Planning Policy (FPP)                          760   
                    Halklarin Demokratik Partisi (HDP) / Baris ve D...    720   
                    Student activist/organizer                            700   
                    Academics/Artists/Intellectuals                       660   
                    Liberation Tigers of Tamil Eelam (LTTE)               600   
                    Employee of foreign/international agent/Entities      420   
                    Desertion                                             420   
                    Land expropriation                                    420   
                    Taliban                                               380   
                    Nepali Congress (NC)                                  300   
                    Southern Cameroon National Council (SCNC)             280   
                    Movement for Democratic Change (MDC)                  240   
                    Sikh militant - Other                                 200   
                    Trade/Labour unionist                                 200   
                    Akali Dal (Badal)/Shiromani Akali Dal (Badal)         200   
                    Union for Democracy and Social Progress (UDPS)        180   
                    Revolutionary Armed Forces of Colombia (FARC)         180   
                    La Fanmi Lavalas/Lavalas Family Party (FL)            160   
                    Bangladesh Nationalist Party (BNP)                    140   
                    Coalition for Unity & Democracy (CUD) - (includ...    120   
                    Kashmir based party                                   120   
                    Militant - other/unspecified                          120   
                    National Democratic Party/Rastriya Prajantra Pa...    120   
                    Hamas                                                 120   
                    Illegal Exit                                          100   
                    Pro-Saudi/Arab alliance militia/group-supporter...    100   
                    Forum for Democratic Change (FDC)                     100   
                    Acción Democrática - AD (Democratic Action)           100   
                    Anglophone minority rights                            100   
                    Peoples Democratic Party (PDP)                         80   
                    Oromo Liberation Front (OLF)                           80   
                    Opposition party/movement                              80   
                    Communist Party of Nepal (Maoist) (CPN-M)              80   
                    Assembly of Progressive National Democrats (RDNP)      60   
                    Awami League (AL)                                      60   
                    Pitit Dessalines                                       60   
                    Struggling People's Organization (OPL)                 60   
                    Primero Justicia (Justice First)                       60   
                    Congress Party                                         60   
                    Anti-Aristide                                          40   
                    Chin

In [26]:
# Table 3a Political Countries

piv=summarize_two_level(dfAggregated,
                        col1_wanted='Aggregated_Category',
                        var_col1_wanted='Political Opinion',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 3a PO countries.csv')

piv


All  Proportion (%)  \
Aggregated_Category Country_Persecution                               
Political Opinion   All                       38740             100   
                    Turkey                     3600               9   
                    Haiti                      2460               6   
                    Venezuela                  1960               5   
                    Eritrea                    1800               5   
                    Burundi                    1740               4   
                    Afghanistan                1680               4   
                    Ethiopia                   1600               4   
                    China                      1520               4   
                    India                      1480               4   
                    Congo, DRC                 1440               4   
                    Syria                      1360               4   
                    Iran                       1200               3   
                    Sudan                      1160               3   
                    Sri Lanka                  1060               3   
                    Yemen                      1000               3   
                    Colombia                    800               2   
                    Cameroon                    700               2   
                    Pakistan                    680               2   
                    Egypt                       660               2   
                    Nigeria                     660               2   
                    Nepal                       660               2   
                    Rwanda                      640               2   
                    Bangladesh                  620               2   
                    Libya                       580               1   
                    Iraq                        540               1   
                    Zimbabwe                    460               1   
                    Somalia                     440               1   
                    Palestine                   440               1   
                    Cuba                        420               1   
                    Azerbaijan                  400               1   
                    Chad                        380               1   
                    Djibouti                    360               1   
                    Nicaragua                   320               1   
                    Uganda                      320               1   
                    Georgia                     280               1   
                    Mexico                      240               1   
                    Angola                      220               1   
                    Lebanon                     200               1   
                    Ukraine                     200               0   
                    Congo (Brazzaville)         180               0   
                    Algeria                     120               0   
                    Saudi Arabia                120               0   
                    Ivory Coast                 120               0   
                    Guinea                      100               0   
                    Kenya                       100               0   
                    Mauritania                   80               0   
                    Korea DPR (North)            80               0   
                    Togo                         60               0   
                    Russia                       60               0   
                    Honduras                     60               0   
                    United States of America     60               0   
                    Gambia                       60               0   
                    El Salvador                  60               0   
                    Vietnam                      40               0   
 

In [27]:
# Table 4 GBV Types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Particular Social Group - Gender-based/Domestic Violence',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 4 GBV types.csv')
piv

All  \
Category                                           Type                                     
Particular Social Group - Gender-based/Domestic... All                              14300   
                                                   Domestic violence                 5380   
                                                   Female - Other                    2900   
                                                   Non-domestic sexual violence      2380   
                                                   Forced marriage                   1940   
                                                   Female Genital Mutilation (FGM)   1840   
                                                   Male - Other                       340   
                                                   Honor Crime                        320   
                                                   Child Abuse                        120   
                                                   Widowhood rites                     80   
                                                   Female - Honour killing             60   
                                                   Forced prostitution                 40   
                                                   Sexual harassment                   20   

                                                                                    Proportion (%)  \
Category                                           Type                                              
Particular Social Group - Gender-based/Domestic... All                                         100   
                                                   Domestic violence                            38   
                                                   Female - Other                               20   
                                                   Non-domestic sexual violence                 17   
                                                   Forced marriage                              14   
                                                   Female Genital Mutilation (FGM)              13   
                                                   Male - Other                                  2   
                                                   Honor Crime                                   2   
                                                   Child Abuse                                   1   
                                                   Widowhood rites                               1   
                                                   Female - Honour killing                       0   
                                                   Forced prostitution                           0   
                                                   Sexual harassment                             0   

                                                                                    Recognition Rate (%)  
Category                                           Type                                                   
Particular Social Group - Gender-based/Domestic... All                                                70  
                                                   Domestic violence                                  72  
                                                   Female - Other                                     78  
                                                   Non-domestic sexual violence                       73  
                                                   Forced marriage                                    75  
                                                   Female Genital Mutilation (FGM)                    47  
                                                   Male - Other                                       53  
                                                   Honor Crime                                        73  
                                                   Child Abuse                           

In [28]:
# Table 4a GBV Countries
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Particular Social Group - Gender-based/Domestic Violence',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 4a GBV countries.csv')
piv

All  \
Category                                           Country_Persecution               
Particular Social Group - Gender-based/Domestic... All                       14300   
                                                   Nigeria                    2860   
                                                   Haiti                      1200   
                                                   Iran                        900   
                                                   India                       540   
                                                   Mexico                      440   
                                                   Congo, DRC                  360   
                                                   Pakistan                    340   
                                                   Afghanistan                 320   
                                                   Uganda                      260   
                                                   Kenya                       240   
                                                   Somalia                     240   
                                                   Syria                       240   
                                                   Iraq                        220   
                                                   Cameroon                    220   
                                                   Djibouti                    220   
                                                   Guinea                      200   
                                                   Burundi                     200   
                                                   Jamaica                     180   
                                                   Sudan                       160   
                                                   Saudi Arabia                160   
                                                   Colombia                    160   
                                                   Bangladesh                  160   
                                                   Lebanon                     160   
                                                   Ukraine                     160   
                                                   Algeria                     140   
                                                   Jordan                      140   
                                                   China                       140   
                                                   Turkey                      140   
                                                   Egypt                       140   
                                                   Chad                        120   
                                                   Yemen                       120   
                                                   Hungary                     120   
                                                   Senegal                     100   
                                                   Ethiopia                    100   
                                                   Sri Lanka                   100   
                                                   Bahamas                     100   
                                                   Mali                        100   
                                                   Georgia                     100   
                                                   Rwanda                      100   
                                                   Zimbabwe                     80   
                                                   Ivory Coast                  80   
                                                   El Salvador                  80   
                                                   Palestine                    80   
                                                   Ghana                        80   
                                     

In [29]:
# Table 5: SOGIE Types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Particular Social Group - Sexual Orientation',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv,TableDir + 'Table 5 SOGIE types.csv')

piv


All  \
Category                                     Type                                         
Particular Social Group - Sexual Orientation All                                  12760   
                                             Gay                                   5800   
                                             Bisexual                              3620   
                                             Lesbian                               2720   
                                             Varied/Other                           600   
                                             Transgender                             80   
                                             Imputed Sexual Orientation              60   
                                             Family of / Related to LGBTQ person     60   

                                                                                  Proportion (%)  \
Category                                     Type                                                  
Particular Social Group - Sexual Orientation All                                             100   
                                             Gay                                              45   
                                             Bisexual                                         28   
                                             Lesbian                                          21   
                                             Varied/Other                                      5   
                                             Transgender                                       1   
                                             Imputed Sexual Orientation                        0   
                                             Family of / Related to LGBTQ person               0   

                                                                                  Recognition Rate (%)  
Category                                     Type                                                       
Particular Social Group - Sexual Orientation All                                                    77  
                                             Gay                                                    79  
                                             Bisexual                                               68  
                                             Lesbian                                                84  
                                             Varied/Other                                           73  
                                             Transgender                                            97  
                                             Imputed Sexual Orientation                             74  
                                             Family of / Related to LGBTQ person                    73

In [30]:
# Table 5a: SOGIE countries

piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Particular Social Group - Sexual Orientation',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)

save_to_csv(piv,TableDir+'Table 5a SOGIE countries.csv')
piv



All  \
Category                                     Country_Persecution               
Particular Social Group - Sexual Orientation All                       12760   
                                             Nigeria                    3420   
                                             Uganda                      680   
                                             Jamaica                     580   
                                             Cameroon                    500   
                                             Ghana                       420   
                                             Turkey                      420   
                                             Kenya                       380   
                                             India                       360   
                                             Ukraine                     360   
                                             Pakistan                    340   
                                             Bahamas                     340   
                                             Iran                        340   
                                             Mexico                      300   
                                             Haiti                       300   
                                             Rwanda                      220   
                                             Senegal                     220   
                                             Russia                      200   
                                             Barbados                    180   
                                             Albania                     180   
                                             Tanzania                    180   
                                             Algeria                     120   
                                             Congo, DRC                  100   
                                             Colombia                    100   
                                             Venezuela                   100   
                                             Zimbabwe                    100   
                                             Mali                        100   
                                             Guinea                       80   
                                             Lebanon                      80   
                                             Ivory Coast                  60   
                                             Saint Lucia                  60   
                                             Saudi Arabia                 60   
                                             Georgia                      60   
                                             Mongolia                     60   
                                             Brazil                       60   
                                             Sri Lanka                    60   
                                             Bangladesh                   60   
                                             Togo                         60   
                                             Jordan                       60   
                                             Egypt                        60   
                                             Saint Vincent                40   
                                             Antigua and Barbuda          40   
                                             Chad                         40   
                                             Gambia                       40   
                                             Tunisia                      40   
                                             Hungary                      40   
                                             Ethiopia                     40   
                                             Dominican Republic           40   
                                             Burundi                     

In [31]:
# Table 6 No nexus: types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Aggregated_Category',
                        var_col1_wanted='No Nexus',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
#If you want to save
save_to_csv(piv, TableDir + 'Table 6 No Nexus types.csv')
piv


All  \
Aggregated_Category Type                                                        
No Nexus            All                                                 25580   
                    Witness/Victim of Organized crime                    5720   
                    Personal vendetta/Family feud                        4480   
                    Witness/Victim of Common crime                       3760   
                    Witness/Victim of State agents                       2560   
                    Varied/Other                                         2420   
                    Witness/Victim of radical fundamentalist group       1960   
                    Witness/Victim of Guerrilla/Rebels                   1320   
                    Fear is unspecified/unclear                          1200   
                    Generalized risk                                      980   
                    Land dispute                                          420   
                    Witness/Victim of Paramilitary                        400   
                    Blood feud                                            160   
                    Witness/Victim of Hezbollah                           160   
                    Forced recruitment by gangs                           160   
                    Returnee/Expatriate                                   140   
                    Economic migrant                                      140   
                    Witness/Victim of Radical fundamentalist group ...    140   
                    Forced recruitment/collusion by Guerrilla             120   
                    Witness/Victim of pro-Houthi groups                   120   
                    Health Care/Medical condition                         100   
                    Forced recruitment/collusion by Paramilitary           80   
                    Witness/Victim of Radical Fundamentalist Group ...     80   
                    Witness/Victim of Mungiki                              60   
                    Land dispute/Appropriation                             60   
                    Witness/Victim of Radical Fundamentalist Group ...     60   
                    Witness/Victim of Chimeres                             60   
                    Witness/Victim of Organized crime - MS-13              40   
                    Witness/Victim of Organized Crime - Other              40   
                    Witness/Victim of Organized Crime - Clan Usuga/...     40   
                    Witness/Victim of State agents and Guerrilla/Re...     40   
                    Witness/Victim of Organized crime - M-18               20   
                    Witness/Victim of militia                              20   
                    Forced recruitment by gangs - MS-13                    20   
                    Witness/Survivor of Genocide                           20   
                    Witness/Victim of Ambazonia Defence Forces/Amba...     20   
                    Witness/Victim of Guerrilla/Rebels - Other group       20   
                    Forced recruitment by gangs - M-18                     20   

                                                                        Proportion (%)  \
Aggregated_Category Type                                                                 
No Nexus            All                                                            100   
                    Witness/Victim of Organized crime                               22   
                    Personal vendetta/Family feud                                   18   
                    Witness/Victim of Common crime                                  15   
                    Witness/Victim of State agents                                  10   
                    Varied/Other                                                     9   
                    Witness/Victim of radical fundamentalist group                   8   
                    Wit

In [32]:
# Table 6a No nexus: countries
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Aggregated_Category',
                        var_col1_wanted='No Nexus',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 6a No Nexus countries.csv')
piv

All  Proportion (%)  \
Aggregated_Category Country_Persecution                               
No Nexus            All                       25580             100   
                    Haiti                      3700              15   
                    Mexico                     3220              13   
                    Colombia                   2280               9   
                    Nigeria                    1860               7   
                    India                      1280               5   
                    Pakistan                   1020               4   
                    Somalia                     800               3   
                    El Salvador                 800               3   
                    Iraq                        500               2   
                    Honduras                    480               2   
                    Bangladesh                  440               2   
                    Afghanistan                 420               2   
                    Lebanon                     400               2   
                    Venezuela                   400               2   
                    Sri Lanka                   380               1   
                    Yemen                       360               1   
                    Congo, DRC                  340               1   
                    Albania                     320               1   
                    Syria                       280               1   
                    Libya                       280               1   
                    Palestine                   260               1   
                    Guatemala                   240               1   
                    China                       240               1   
                    Iran                        240               1   
                    Jordan                      240               1   
                    Algeria                     220               1   
                    Kenya                       200               1   
                    Jamaica                     200               1   
                    United States of America    180               1   
                    Egypt                       180               1   
                    Cameroon                    140               1   
                    Burundi                     140               1   
                    Angola                      140               1   
                    Philippines                 120               0   
                    Bahamas                     100               0   
                    Ghana                       100               0   
                    Georgia                     100               0   
                    Brazil                      100               0   
                    Rwanda                      100               0   
                    Chad                        100               0   
                    Peru                         80               0   
                    Uganda                       80               0   
                    Ethiopia                     80               0   
                    Ukraine                      80               0   
                    Turkey                       80               0   
                    Kosovo                       60               0   
                    Saudi Arabia                 60               0   
                    Saint Vincent                60               0   
                    Nepal                        60               0   
                    Cuba                         60               0   
                    Mali                         60               0   
                    Eritrea                      60               0   
                    Dominican Republic           60               0   
                    Senegal                      60               0   
 

In [33]:
# Table 7 Religion: types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Religion',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 7 Religion types.csv')
piv

All  Proportion (%)  \
Category Type                                                                   
Religion All                                            18580             100   
         Christian                                       5080              27   
         Varied/Other                                    3400              18   
         Apostasy                                        2080              11   
         Ahmadi (unspecified)                            1100               6   
         Muslim - Sunni                                   960               5   
         Christian - Coptic                               860               5   
         Shia Muslim                                      700               4   
         Muslim - Shia                                    680               4   
         Muslim                                           480               3   
         Christian - Unregistered/underground church      400               2   
         Chieftaincy Issues                               320               2   
         Atheist/Agnostic/non practicing                  320               2   
         Pentecostal                                      280               2   
         Traditional                                      260               1   
         Alevi                                            260               1   
         Rituals/Witchcraft/Traditional practices         220               1   
         Inter-faith Marriage/Relationship                200               1   
         Bahai                                            180               1   
         Hindu                                            180               1   
         Ahmadi (Lahori)                                  160               1   
         Buddhist                                         140               1   
         Sikh                                             140               1   
         Sunni - majority or minority sect                120               1   
         Christian - Other                                100               1   
         Christian - Church of Almighty God                80               0   
         Ahmadi (Qadiani)                                  80               0   
         Fetish Priest issues                              40               0   
         Erfan Keyhani/Erfan Halgheh/Interuniversalism     40               0   
         Sunni Muslim                                      40               0   
         Sufi                                              40               0   
         Dera Sasha Sauda                                  20               0   
         Ahmadi                                            20               0   
         Shia - majority or minority sect                  20               0   
         Jehovah’s Witness                                 20               0   

                                                        Recognition Rate (%)  
Category Type                                                                 
Religion All                                                              78  
         Christian                                                        75  
         Varied/Other                                                     70  
         Apostasy                                                         97  
         Ahmadi (unspecified)                                             98  
         Muslim - Sunni                                                   93  
         Christian - Coptic                                               97  
         Shia Muslim                                                      71  
         Muslim - Shia                                                    68  
         Muslim                                                           65  
         Christian - Unregistered/underground church                      71  
         Chieftaincy Issues      

In [34]:
# Table 7 Religion: countries
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Religion',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 7a Religion countries.csv')
piv

All  Proportion (%)  Recognition Rate (%)
Category Country_Persecution                                                  
Religion All                       18580             100                    78
         Iran                       4160              22                    95
         Pakistan                   3020              16                    82
         China                      2140              12                    62
         Nigeria                    1360               7                    33
         Iraq                       1180               6                    89
         Syria                      1100               6                    97
         Egypt                      1000               5                    97
         India                       540               3                    37
         Eritrea                     460               2                    87
         Turkey                      360               2                    82
         Bangladesh                  340               2                    78
         Somalia                     320               2                    68
         Yemen                       240               1                    96
         Algeria                     200               1                    70
         Afghanistan                 180               1                    88
         Lebanon                     160               1                    55
         Saudi Arabia                160               1                    94
         Sri Lanka                   140               1                    76
         Jordan                      140               1                    66
         Russia                      100               0                    81
         Ghana                        80               0                    21
         Sudan                        60               0                    90
         Haiti                        60               0                    28
         Libya                        60               0                    96
         Palestine                    60               0                    84
         Ethiopia                     40               0                    64
         Angola                       40               0                    55
         Congo, DRC                   40               0                    54
         Morocco                      40               0                    52
         Kazakhstan                   40               0                    89
         Ukraine                      40               0                    47
         Central African Rep          40               0                    97
         Tunisia                      40               0                    40
         Vietnam                      20               0                    30
         Cameroon                     20               0                    54
         Azerbaijan                   20               0                    86
         Kenya                        20               0                    59
         Senegal                      20               0                    14
         Indonesia                    20               0                    54
         United States of America     20               0                     0
         Mexico                       20               0                    30
         Uzbekistan                   20               0                    73
         Cuba                         20               0                    55
         Uganda                       20               0                    74
         Mali                         20               0                    37
         Liberia                      20               0                    50
         Nepal                        20               0                    88
         Guinea                       20               0                    77
        

In [35]:
# Table 8 Race: types
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Race/Ethnicity/Nationality',
                        col2_wanted='Type',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 8 Race types.csv')

piv


All  \
Category                   Type                                       
Race/Ethnicity/Nationality All                                13580   
                           Varied/Other                        3060   
                           Roma                                2520   
                           Tutsi                               1080   
                           Kurd                                 920   
                           Alevi Kurd                           820   
                           Tamil-North/East/Central             760   
                           Palestinian                          460   
                           Tibetan                              400   
                           Madhiban/Midgan/Gaboye               360   
                           Oromo                                340   
                           Amhara                               240   
                           Mixed Marriage/Relationship          240   
                           Hazaras                              220   
                           Jiberti                              180   
                           Asharaf/Ashraf                       180   
                           Sheekhaal/Sheikhal/Shikal            160   
                           Caste issues                         140   
                           Muslim Ouighor                       120   
                           Tamil-Colombo                         80   
                           Bedun                                 80   
                           Toubou (Gorane)                       80   
                           Afro-Colombian                        80   
                           Uighur                                80   
                           Tuni                                  60   
                           Reer Hamar                            60   
                           Tomal/Tumal                           60   
                           Ogoni/Ogboni                          40   
                           Tigrean                               40   
                           Tutsi/Banyamulenge                    40   
                           Darod/Marehan/Majeerteen              40   
                           Tamil-speaking Muslims                40   
                           Sunni related tribe/clan              40   
                           Indigenous                            40   
                           Russian                               40   
                           Hawiye/Ajuraan/Habar Gebir/Abgal      40   
                           Ossetian                              40   
                           Bajuni                                40   
                           Ethnic Eritrean                       40   
                           Other minority tribe/clan             40   
                           Mixed Hutu-Tutsi                      40   
                           Chinese                               20   
                           Barawan/Bravan/Reer Brava/Benadir     20   
                           Nubian                                20   
                           Digil/Rahanweyn/Bagadi/Garre          20   
                           Hutu                                  20   
                           Yibir                                 20   
                           Azeri/Azerbaijani                     20   
                           Jiberti/Jeberti                       20   
                           (Anglophones)                         20   
                           Jewish                                20   
                           Dir/Issa/Gadabuursi/Madahweyn         20   
                           Tajik                                 20   
                           Kabyle                                20   
                           Uzbek                                 20   
                 

In [36]:
# Table 8a Race: countries
piv=summarize_two_level(dfAggregated,
                        col1_wanted='Category',
                        var_col1_wanted='Race/Ethnicity/Nationality',
                        col2_wanted='Country_Persecution',
                        row_nums=500,
                        round_nearest=20)
save_to_csv(piv, TableDir + 'Table 8a Race countries.csv')

piv

All  Proportion (%)  \
Category                   Country_Persecution                               
Race/Ethnicity/Nationality All                       13580             100   
                           Turkey                     1580              12   
                           Somalia                    1320              10   
                           Hungary                    1100               8   
                           Burundi                    1080               8   
                           Sri Lanka                   920               7   
                           Ethiopia                    820               6   
                           China                       600               4   
                           Romania                     460               3   
                           Slovakia                    400               3   
                           Palestine                   380               3   
                           Lebanon                     300               2   
                           Ukraine                     280               2   
                           Afghanistan                 260               2   
                           India                       240               2   
                           Eritrea                     220               2   
                           Yemen                       200               1   
                           Czech Republic              180               1   
                           Iran                        160               1   
                           Nigeria                     160               1   
                           Saudi Arabia                140               1   
                           Iraq                        140               1   
                           Syria                       140               1   
                           Libya                       120               1   
                           Djibouti                    120               1   
                           Chad                        120               1   
                           Sudan                       100               1   
                           Pakistan                    100               1   
                           Russia                      100               1   
                           Jordan                      100               1   
                           Haiti                       100               1   
                           Congo, DRC                  100               1   
                           Kuwait                      100               1   
                           Poland                       80               1   
                           Colombia                     80               1   
                           Rwanda                       80               1   
                           Croatia                      80               1   
                           Egypt                        60               1   
                           Mauritania                   60               1   
                           Bulgaria                     60               0   
                           Cameroon                     60               0   
                           Algeria                      40               0   
                           South Africa                 40               0   
                           Georgia                      40               0   
                           Mexico                       40               0   
                           Bangladesh                   40               0   
                           Venezuela                    40               0   
                           Israel                       40               0   
                           Guinea                       40               0   
                           United States of America     20               0   

In [37]:
# For Chart 4/4a: aggregated cats per year
years_dict={}
for x in range (2013,2022):  #note the plus one here
    #print(x)
    piv=summarize_two_level(dfAggregated,
                            col1_wanted='Decision_Date',
                            var_col1_wanted=x,
                            col2_wanted='Aggregated_Category',
                            row_nums=500,
                            round_nearest=20)
    piv=piv.droplevel(0)
    piv=piv['All']
    pivdict = piv.to_dict()
    years_dict[x]=pivdict

#print(years_dict)
piv2=pd.DataFrame(years_dict)

piv2.to_csv(TableDir+'for chart 4 and 4a.csv')

piv2

,2013,2014,2015,2016,2017,2018,2019,2020,2021
All,3920,7040,7540,8960,12360,12640,20980,13280,26320
Particular Social Group,1200,2180,2320,2520,3240,2900,5080,3660,8160
Political Opinion,1140,1960,2200,3120,4880,4640,7520,4700,8600
No Nexus,860,1400,1320,1520,2140,2660,4660,3280,7720
Religion,680,1200,1120,1360,1740,1560,3580,2400,4940
Race/Ethnicity/Nationality,460,860,1020,1380,1640,1500,2260,1480,2960
No Category Provided,160,180,200,200,360,1000,1200,520,680
